# Visualization Example

In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [ ]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

: 

In [ ]:
import math
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from src.pipeline.flows.main import main_flow
from src.visualization.interactive_play_selector import create_interactive_play_selector

: 

In [ ]:
_ = main_flow(max_games=3, max_plays=3)

: 

In [ ]:
df_plays_all = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_tracking_display_all = pd.read_csv(f"{DIR}/data/outputs/tracking_display.csv")
df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

: 

In [ ]:
create_interactive_play_selector(
    df_plays_all,
    df_tracking_display_all,
    df_areas_all,
    continuous_update=True
)

: 

: 